In [21]:
import re,sys,time,os,sh,time,json
import argparse
import pandas as pd

In [22]:
# 设置工作目录
workdir = os.path.join('/home/dongjc/work/mtd/tools/') 
workdir_blast = os.path.join(workdir,'blast')
# print(workdir_blast)

In [24]:
# 构建的蛋白库路径
mt_prot_db = os.path.join(workdir_blast,' db_prot/Myceliophthora_thermophila_ATCC_42464')
nc_prot_db = os.path.join(workdir_blast, 'db_prot/Neurospora_crassa_OR74A') 
all_prot_db = os.path.join(workdir_blast, 'db_prot/All_species')

# 比对嗜热毁丝霉蛋白序列
def blast_prot(blast_input_path, blast_output_path, db_prot,  evalue):
    """
    Description: 比对嗜热毁丝霉蛋白序列
    Args:
        blast_input_path: 输入文件路径
        blast_output_path: 输出文件路径
        db_prot: 菌种数据库
        evalue: evalue值
    """
    try:
        if not os.path.exists(blast_input_path):
            raise Exception("Error: The input blast file does not exist. Please check again.")
        
        cmd = "blastp -query {} -out {} -db {} -outfmt 6 -evalue {}".format(blast_input_path, blast_output_path, db_prot, evalue)

        # os.system("blastp -query "+blast_input_path+" -db "+ species_db_prot+" "+ blast_output_path +" -outfmt 6 -out -evalue 1e-6")
        os.system(cmd)
    
    except Exception as e:
        print(e)
        sys.exit(1)

In [25]:
blast_prot('blast_input.txt', 'blast_output.txt', all_prot_db, 1e-6)

In [27]:
df_blast = pd.read_csv('blast_output.txt', sep='\t', header=None)
df_blast.columns = ['query_id', 'subject_id', 'pct_identity', 'aln_length', 'mismatches', 'gap_opens', 'q_start', 'q_end', 's_start', 's_end', 'evalue', 'bit_score']
df_blast.head()

,query_id,subject_id,pct_identity,aln_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue,bit_score
0,NP_013214.1-YLR113W-HOG1,XP_003665798.1,82.000,350,62,1,6,354,3,352,0.000000e+00,622.0
1,NP_013214.1-YLR113W-HOG1,XP_962163.2,82.759,348,59,1,6,353,3,349,0.000000e+00,619.0
2,NP_013214.1-YLR113W-HOG1,XP_011395162.1,47.198,339,164,4,17,341,14,351,1.940000e-101,305.0
3,NP_013214.1-YLR113W-HOG1,XP_011395161.1,47.198,339,164,4,17,341,14,351,1.940000e-101,305.0
4,NP_013214.1-YLR113W-HOG1,XP_003665446.1,47.198,339,164,4,17,341,15,352,3.020000e-101,305.0


In [ ]:
# 提取标识符(蛋白ID)
ae_list = list(df_blast.iloc[:, 1])

# 正则定义
regular = '>(.*?)\s' 

# 提取同源序列后合并成fasta文件的地址
extract_output_path = os.path.join(workdir, 'extract_output.fasta')

In [ ]:
# 根据标识符，从菌种蛋白序列中提取相关序列，合并到一个新的fasta
def read_prot_sequences(species_prot, extract_output_path):

    if os.path.exists(species_prot) == False:
        print("Error: The input file does not exist. Please check again.")
        sys.exit(1)
        
    with open(species_prot, 'r') as f:
        fasta = f.readlines() 
        fasta_dict = {}

        for line in fasta:
            if line.startswith('>'):
                name = line  
                fasta_dict[name] = ''  
                continue
            fasta_dict[name] += line
   
    count = 0
    sequence = []

    for ano,sequences in fasta_dict.items():
        for ae in ae_list:
            ae_name = re.search(regular, ano).group(1)  #  re.search匹配，group获取匹配的内容
            if ae == ae_name:
                sequence.append([ano, sequences])
                count += 1

    with open(extract_output_path, 'w') as r:
        for line in sequence:
            r.writelines(line)
        
    return sequence

In [ ]:
# 匹配所有的菌种蛋白序列，提取序列并合并
merger_output = read_prot_sequences(mt_prot_db, 'extract_output.fasta')

In [ ]:
# mafft对齐完输出的文件地址
mafft_output_path = os.path.join(workdir + "output/mafft.fasta")

In [ ]:
# mafft对齐
def mafft(extract_output, mafft_output):

    if os.path.exists(extract_output) == False:
        print("Error: The input file does not exist. Please check again.")
        sys.exit(1)

    os.system("mafft --auto "+extract_output+" > "+mafft_output)

In [ ]:
# 对齐
mafft_output = mafft(extract_output_path, mafft_output_path)

In [ ]:
# fasttree运行完输出的文件地址
fasttree_prot_output_path = os.path.join(workdir + "output/prot_output.tre")
fasttree_nucl_output_path = os.path.join(workdir + "output/nucl_output.tre")

In [ ]:
# fasttree对核酸序列
def fasttree_nucl(mafft_output):

    if os.path.exists(mafft_output) == False:
        print("Error: The input file does not exist. Please check again.")
        sys.exit(1)

    os.system("FastTree -nt -gtr -gamma "+mafft_output+" > "+fasttree_nucl_output_path)

In [ ]:
# fasttree对蛋白序列
def fasttree_prot(mafft_output):

    if os.path.exists(mafft_output) == False:
        print("Error: The input file does not exist. Please check again.")
        sys.exit(1)

    os.system("FastTree "+mafft_output+" > "+fasttree_prot_output_path)

In [ ]:
fasttree_prot_output = fasttree_prot(mafft_output_path)